In [3]:
!pip install nltk

In [4]:
import nltk
nltk.download('punkt')

from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /home/emmanuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
class Node:
    def __init__(self, item):
        self.item = item
        self.next = None

class LL:
    def __init__(self):
        self.head = None
    def add_node(self,val):
        newNode = Node(val)
        if not self.head:
            self.head = newNode
            return
        temp = self.head
        while temp:
            if val == temp.item:
                return
            temp = temp.next
        if self.head.item > val:
            newNode.next = self.head
            self.head = newNode
            return
        temp = self.head
        while temp and temp.next:
            if val < temp.next.item:
                break
            temp = temp.next
        newNode.next = temp.next
        temp.next = newNode
                    
    def print_list(self):
        tmp = self.head
        while tmp:
            print(tmp.item, end=' ')
            tmp = tmp.next

In [6]:
import pickle

with open("inverted_index.pkl", "rb") as f:
    inverted_index= pickle.load(f)

with open("positional_index.pkl", "rb") as f:
    positional_index = pickle.load(f)

with open("docs.pkl", "rb") as f:
    docs = pickle.load(f)

In [10]:
import re
def evalPosQuery(term1, term2, k):
    answer = []
    def posIntersect(p1, p2, k, doc):
        l = []
        pp1 = p1.head
        pp2 = p2.head
        while pp1 is not None:
            while pp2 is not None:
                if abs(pp1.item - pp2.item) <= k:
                    l.append(pp2.item)
                elif pp2.item > pp1.item:
                    break
                pp2 = pp2.next
            while l != [] and abs(l[0] - pp1.item) > k:
                l.pop(0)
            for ps in l:
                answer.append([doc, pp1.item, ps])
            pp1 = pp1.next
    if positional_index.get(term1) and positional_index.get(term2):
        for doc in docs:
            if positional_index[term1].get(doc) and positional_index[term2].get(doc):
                posIntersect(positional_index[term1][doc], positional_index[term2][doc], k, doc)
            else:
                continue
    return answer

In [ ]:
import tkinter as tk
from tkinter import ttk
import re
from random import choice  # Assuming you have 'choice' from the random module
# Import necessary functions from your code, e.g., evalPosQuery, ps, etc.

def search_proximity():
    # Get the proximity query from the user
    query = entry.get()

    tkns = query.split(' ')
    answer = evalPosQuery(ps.stem(tkns[0]), ps.stem(tkns[1]), int(re.search(r'\d+', tkns[2]).group()))

    table = ttk.Treeview(root, columns=('doc', 'first', 'second'), show='headings')
    table.heading('doc', text='Document ID')
    table.heading('first', text='First word index')
    table.heading('second', text='Second word index')
    table.grid(row=2, column=0, columnspan=2, sticky="nsew")

    for i in range(len(answer)):
        doc = answer[i][0]
        first_idx = answer[i][1]
        second_idx = answer[i][2]
        data = (doc, first_idx, second_idx)
        table.insert(parent='', index=i, values=data)

    # Insert a default value at the end
    table.insert(parent='', index=tk.END, values=('XXXXX', 'YYYYY', 'ZZZZZ'))

# Create the main Tkinter window
root = tk.Tk()
root.title("Proximity Query Search")

# Create an input entry widget
entry = tk.Entry(root, width=30)
entry.grid(row=0, column=0, padx=10, pady=10, columnspan=2)

# Create a button to trigger the search
search_button = tk.Button(root, text="Search", command=search_proximity)
search_button.grid(row=1, column=0, padx=10, pady=10, columnspan=2)

# Create a label to display the output
output_label = tk.Label(root, text="")
output_label.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

# Set row and column weights for resizing
root.grid_rowconfigure(2, weight=1)
root.grid_columnconfigure(0, weight=1)
root.grid_columnconfigure(1, weight=1)

# Start the Tkinter event loop
root.mainloop()